In [48]:

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Import Packages
import pandas as pd
import holoviews as hv
import hvplot.pandas
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression

In [49]:
dv_url = (
    "https://www.ncei.noaa.gov/access/services/data/v1?"
    "dataset=daily-summaries&"
    "stations=USC00042319&"
    "dataTypes=TOBS&"
    "startDate=2000-01-01&"
    "endDate=2024-12-31&"
    "units=metric" # this here will get me the data in celsius
)

dv_url



'https://www.ncei.noaa.gov/access/services/data/v1?dataset=daily-summaries&stations=USC00042319&dataTypes=TOBS&startDate=2000-01-01&endDate=2024-12-31&units=metric'

In [46]:


# Download the climate data using the url
dv_df = pd.read_csv(
    dv_url,
    index_col='DATE',
    parse_dates=True,
    na_values=['NaN']
)

# Check if the data was downloaded
dv_df.head()

,STATION,TOBS
DATE,,
2000-01-01,USC00042319,3.3
2000-01-02,USC00042319,12.8
2000-01-03,USC00042319,1.7
2000-01-04,USC00042319,3.3
2000-01-05,USC00042319,1.1


In [40]:

# selecting the column for temperature at time of observation and 
# reassign to the dataframe.
dv_df = dv_df[['TOBS']]
dv_df

,TOBS
DATE,
2000-01-01,3.3
2000-01-02,12.8
2000-01-03,1.7
2000-01-04,3.3
2000-01-05,1.1
...,...
2025-09-23,NaN
2025-09-24,NaN
2025-09-25,NaN


In [47]:
# Resample data to obtain the mean annual temperature.
ann_dv_df = dv_df.resample('YS').mean()
ann_dv_df.head()

TypeError: agg function failed [how->mean,dtype->object]

In [42]:
# Plot the annual data interactively
ann_dv_df_plot = ann_dv_df.hvplot(y='TOBS',
    title=' Mean Annual Temperature (C) of \nDeath Valley National Park, USA',
    xlabel='Year',
    ylabel='Mean Annual Temperature (Celsius)'
    )

ann_dv_df_plot

:Curve   [DATE]   (TOBS)

In [ ]:

# Drop NaN values
ann_dv_df_new = ann_dv_df.dropna()

# Reshape 'Year' column to be a 2D array for scikit-learn
X = ann_dv_df_new.index.values.reshape(-1,1)
Y = ann_dv_df_new['TOBS'].values

# Create and fit the linear regression model
model = LinearRegression()
model.fit(X, Y)

# Get the slope to understand the average warming per year
slope = model.coef_[0]

# Print the results
print(("From 2020 to 2024, Back Bay NWF has warmed an average of {}"
      # " \N{DEGREE SIGN}C per year.").format(round(slope, 3)))